Book Recommender System using Clustering | Collaborative Based

In [269]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy
import sys
import pickle
import os
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse import csr_matrix

In [201]:
print(scipy.__version__)
print(sys.executable)

1.4.1
c:\Users\gbemi\anaconda3\envs\my_env\python.exe


In [202]:
books = pd.read_csv('bkdata/BX-Books.csv', sep=";", on_bad_lines='skip', encoding='latin-1', low_memory=False)

In [203]:
books.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...


In [204]:
books.rename(columns={
    "Book-Title": "title",
    "Book-Author": "author",
    "Year-Of-Publication": "year",
    "Publisher": "publisher",
    "Image-URL-L": "img_url"}, inplace=True)

In [205]:
#fill missing values in 'title' and 'publisher'
books['title'] = books['title'].fillna('')
books['publisher'] = books['publisher'].fillna('')

In [206]:
books.shape

(271360, 8)

In [207]:
books.columns

Index(['ISBN', 'title', 'author', 'year', 'publisher', 'Image-URL-S',
       'Image-URL-M', 'img_url'],
      dtype='object')

In [208]:
print(books.columns)


Index(['ISBN', 'title', 'author', 'year', 'publisher', 'Image-URL-S',
       'Image-URL-M', 'img_url'],
      dtype='object')


In [209]:
books = books[['ISBN', 'title', 'author', 'year', 'publisher',
       'Image-URL-S', 'Image-URL-M', 'img_url']]

In [210]:
users = pd.read_csv('bkdata/BX-Users.csv', sep=";", on_bad_lines='skip', encoding='latin-1', low_memory=False)

In [211]:
users.head()

,User-ID,Location,Age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",NaN
3,4,"porto, v.n.gaia, portugal",17.0
4,5,"farnborough, hants, united kingdom",NaN


In [212]:
ratings = pd.read_csv('bkdata/BX-Book-Ratings.csv', sep=";", on_bad_lines='skip', encoding='latin-1', low_memory=False)

In [213]:
print(books.shape)
print(users.shape)
print(ratings.shape)

(271360, 8)
(278858, 3)
(1149780, 3)


In [214]:
ratings.head()

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


In [215]:
ratings.rename(columns={
    "User-ID": "user_id",
    "Book-Rating": "rating"}, inplace=True)

In [216]:
ratings.head()

,user_id,ISBN,rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


In [217]:
ratings['user_id'].value_counts()

11676     13602
198711     7550
153662     6109
98391      5891
35859      5850
          ...  
116180        1
116166        1
116154        1
116137        1
276723        1
Name: user_id, Length: 105283, dtype: int64

In [218]:
ratings['user_id'].unique().shape

(105283,)

In [219]:
x = ratings['user_id'].value_counts() > 200

In [220]:
x[x].shape

(899,)

In [221]:
y = x[x].index

In [222]:
y

Int64Index([ 11676, 198711, 153662,  98391,  35859, 212898, 278418,  76352,
            110973, 235105,
            ...
            260183,  73681,  44296, 155916,   9856, 274808,  28634,  59727,
            268622, 188951],
           dtype='int64', length=899)

In [223]:
ratings = ratings[ratings['user_id'].isin(y)]

In [224]:
ratings.head()

,user_id,ISBN,rating
1456,277427,002542730X,10
1457,277427,0026217457,0
1458,277427,003008685X,8
1459,277427,0030615321,0
1460,277427,0060002050,0


In [225]:
ratings.shape

(526356, 3)

In [226]:
ratings_with_books = ratings.merge(books, on ="ISBN")

In [227]:
ratings_with_books.head()

,user_id,ISBN,rating,title,author,year,publisher,Image-URL-S,Image-URL-M,img_url
0,277427,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,http://images.amazon.com/images/P/002542730X.0...,http://images.amazon.com/images/P/002542730X.0...
1,3363,002542730X,0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,http://images.amazon.com/images/P/002542730X.0...,http://images.amazon.com/images/P/002542730X.0...
2,11676,002542730X,6,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,http://images.amazon.com/images/P/002542730X.0...,http://images.amazon.com/images/P/002542730X.0...
3,12538,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,http://images.amazon.com/images/P/002542730X.0...,http://images.amazon.com/images/P/002542730X.0...
4,13552,002542730X,0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,http://images.amazon.com/images/P/002542730X.0...,http://images.amazon.com/images/P/002542730X.0...


In [228]:
ratings_with_books.shape

(487671, 10)

In [229]:
num_rating = ratings_with_books.groupby('title')['rating'].count().reset_index()

In [230]:
num_rating.head()

,title,rating
0,A Light in the Storm: The Civil War Diary of ...,2
1,Always Have Popsicles,1
2,Apple Magic (The Collector's series),1
3,Beyond IBM: Leadership Marketing and Finance ...,1
4,Clifford Visita El Hospital (Clifford El Gran...,1


In [231]:
num_rating.rename(columns={"rating": "num_of_rating"}, inplace=True)

In [232]:
num_rating.head()

,title,num_of_rating
0,A Light in the Storm: The Civil War Diary of ...,2
1,Always Have Popsicles,1
2,Apple Magic (The Collector's series),1
3,Beyond IBM: Leadership Marketing and Finance ...,1
4,Clifford Visita El Hospital (Clifford El Gran...,1


In [233]:
ratings_with_books.head()

,user_id,ISBN,rating,title,author,year,publisher,Image-URL-S,Image-URL-M,img_url
0,277427,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,http://images.amazon.com/images/P/002542730X.0...,http://images.amazon.com/images/P/002542730X.0...
1,3363,002542730X,0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,http://images.amazon.com/images/P/002542730X.0...,http://images.amazon.com/images/P/002542730X.0...
2,11676,002542730X,6,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,http://images.amazon.com/images/P/002542730X.0...,http://images.amazon.com/images/P/002542730X.0...
3,12538,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,http://images.amazon.com/images/P/002542730X.0...,http://images.amazon.com/images/P/002542730X.0...
4,13552,002542730X,0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,http://images.amazon.com/images/P/002542730X.0...,http://images.amazon.com/images/P/002542730X.0...


In [234]:
final_rating=ratings_with_books.merge(num_rating, on= 'title')

In [235]:
final_rating.head()

,user_id,ISBN,rating,title,author,year,publisher,Image-URL-S,Image-URL-M,img_url,num_of_rating
0,277427,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,http://images.amazon.com/images/P/002542730X.0...,http://images.amazon.com/images/P/002542730X.0...,82
1,3363,002542730X,0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,http://images.amazon.com/images/P/002542730X.0...,http://images.amazon.com/images/P/002542730X.0...,82
2,11676,002542730X,6,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,http://images.amazon.com/images/P/002542730X.0...,http://images.amazon.com/images/P/002542730X.0...,82
3,12538,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,http://images.amazon.com/images/P/002542730X.0...,http://images.amazon.com/images/P/002542730X.0...,82
4,13552,002542730X,0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,http://images.amazon.com/images/P/002542730X.0...,http://images.amazon.com/images/P/002542730X.0...,82


In [236]:
final_rating.shape

(487671, 11)

In [237]:
final_rating = final_rating[final_rating['num_of_rating']>=50]

In [238]:
final_rating.sample(10)

,user_id,ISBN,rating,title,author,year,publisher,Image-URL-S,Image-URL-M,img_url,num_of_rating
40201,204522,0451183665,0,A Case of Need,Michael Crichton,1994,Signet Book,http://images.amazon.com/images/P/0451183665.0...,http://images.amazon.com/images/P/0451183665.0...,http://images.amazon.com/images/P/0451183665.0...,77
8672,214786,0446611212,9,Violets Are Blue,James Patterson,2002,Warner Vision,http://images.amazon.com/images/P/0446611212.0...,http://images.amazon.com/images/P/0446611212.0...,http://images.amazon.com/images/P/0446611212.0...,150
34225,110912,0440206154,0,Red Dragon,Thomas Harris,2000,Dell Publishing Company,http://images.amazon.com/images/P/0440206154.0...,http://images.amazon.com/images/P/0440206154.0...,http://images.amazon.com/images/P/0440206154.0...,151
13911,239594,0971880107,0,Wild Animus,Rich Shapero,2004,Too Far,http://images.amazon.com/images/P/0971880107.0...,http://images.amazon.com/images/P/0971880107.0...,http://images.amazon.com/images/P/0971880107.0...,363
40076,50547,0451181379,0,The Door to December,Dean R. Koontz,1994,Signet Book,http://images.amazon.com/images/P/0451181379.0...,http://images.amazon.com/images/P/0451181379.0...,http://images.amazon.com/images/P/0451181379.0...,93
48896,133689,0446329894,0,If Tomorrow Comes,Sidney Sheldon,1986,Warner Books> C/o Little Br,http://images.amazon.com/images/P/0446329894.0...,http://images.amazon.com/images/P/0446329894.0...,http://images.amazon.com/images/P/0446329894.0...,59
131457,95359,0451204948,0,A Day Late and a Dollar Short,Terry McMillan,2002,Signet Book,http://images.amazon.com/images/P/0451204948.0...,http://images.amazon.com/images/P/0451204948.0...,http://images.amazon.com/images/P/0451204948.0...,53
18180,21014,0380731851,0,Mystic River,Dennis Lehane,2002,HarperTorch,http://images.amazon.com/images/P/0380731851.0...,http://images.amazon.com/images/P/0380731851.0...,http://images.amazon.com/images/P/0380731851.0...,130
6427,156269,0439136369,0,Harry Potter and the Prisoner of Azkaban (Book 3),J. K. Rowling,2001,Scholastic,http://images.amazon.com/images/P/0439136369.0...,http://images.amazon.com/images/P/0439136369.0...,http://images.amazon.com/images/P/0439136369.0...,138
93329,101209,0671042572,0,Jewel (Oprah's Book Club),Bret Lott,1999,Pocket,http://images.amazon.com/images/P/0671042572.0...,http://images.amazon.com/images/P/0671042572.0...,http://images.amazon.com/images/P/0671042572.0...,60


In [239]:
final_rating.shape

(61853, 11)

In [240]:
final_rating.drop_duplicates(['user_id','title'], inplace=True)

In [241]:
final_rating.shape

(59850, 11)

In [242]:
book_pivot = final_rating.pivot_table(columns='user_id', index='title', values = 'rating')

In [243]:
book_pivot

user_id,254,2276,2766,2977,3363,3757,4017,4385,6242,6251,...,274004,274061,274301,274308,274808,275970,277427,277478,277639,278418
title,,,,,,,,,,,,,,,,,,,,,
1984,9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN
1st to Die: A Novel,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2nd Chance,NaN,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,NaN
4 Blondes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
84 Charing Cross Road,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,10.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Year of Wonders,NaN,NaN,NaN,7.0,NaN,NaN,NaN,NaN,7.0,NaN,...,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN
You Belong To Me,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Zen and the Art of Motorcycle Maintenance: An Inquiry into Values,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN


In [245]:
book_pivot.fillna(0, inplace=True)

In [246]:
book_pivot

user_id,254,2276,2766,2977,3363,3757,4017,4385,6242,6251,...,274004,274061,274301,274308,274808,275970,277427,277478,277639,278418
title,,,,,,,,,,,,,,,,,,,,,
1984,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1st to Die: A Novel,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2nd Chance,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4 Blondes,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
84 Charing Cross Road,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,10.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Year of Wonders,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,7.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
You Belong To Me,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Zen and the Art of Motorcycle Maintenance: An Inquiry into Values,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [248]:
book_sparse = csr_matrix(book_pivot)

In [250]:
from sklearn.neighbors import NearestNeighbors
model = NearestNeighbors(algorithm = 'brute')

In [251]:
model.fit(book_sparse)

NearestNeighbors(algorithm='brute')

In [252]:
distance, suggestion = model.kneighbors(book_pivot.iloc[237,:].values.reshape(1,-1), n_neighbors=6)

In [253]:
distance

array([[ 0.        , 68.78953409, 69.5413546 , 72.64296249, 76.83098333,
        77.28518616]])

In [254]:
suggestion

array([[237, 240, 238, 241, 184, 536]], dtype=int64)

In [255]:
for i in range(len(suggestion)):
    print(book_pivot.index[suggestion[i]])

Index(['Harry Potter and the Chamber of Secrets (Book 2)',
       'Harry Potter and the Prisoner of Azkaban (Book 3)',
       'Harry Potter and the Goblet of Fire (Book 4)',
       'Harry Potter and the Sorcerer's Stone (Book 1)', 'Exclusive',
       'The Cradle Will Fall'],
      dtype='object', name='title')


In [256]:
book_pivot.index

Index(['1984', '1st to Die: A Novel', '2nd Chance', '4 Blondes',
       '84 Charing Cross Road', 'A Bend in the Road', 'A Case of Need',
       'A Child Called \It\": One Child's Courage to Survive"',
       'A Civil Action', 'A Cry In The Night',
       ...
       'Winter Solstice', 'Wish You Well', 'Without Remorse',
       'Wizard and Glass (The Dark Tower, Book 4)', 'Wuthering Heights',
       'Year of Wonders', 'You Belong To Me',
       'Zen and the Art of Motorcycle Maintenance: An Inquiry into Values',
       'Zoya', '\O\" Is for Outlaw"'],
      dtype='object', name='title', length=742)

In [257]:
books_name = book_pivot.index

In [ ]:
os.makedirs('artifacts', exist_ok=True)

pickle.dump(model, open('artifacts/model.pk1', 'wb'))
pickle.dump(books_name, open('artifacts/books_name.pk1', 'wb'))
pickle.dump(final_rating, open('artifacts/final_rating.pk1', 'wb'))
pickle.dump(book_pivot, open('artifacts/book_pivot.pk1', 'wb'))

In [259]:
# to check if pickling was successful
model = pickle.load(open('artifacts/model.pk1', 'rb'))
print("Model loaded successfully!")


Model loaded successfully!


In [ ]:
def recommend_book(book_name, n_neighbors=6):
    #to get theindex of the queried book
    book_id = np.where(book_pivot.index == book_name)[0][0]
    
    #one extra neighbor to account for the query book itself
    distances, suggestion = model.kneighbors(book_pivot.iloc[book_id, :].values.reshape(1, -1), n_neighbors=n_neighbors+1)
    rec_indices = suggestion[0]
    
    recommended_titles = []
    for idx in rec_indices:
        #skip the book itself
        if idx == book_id:
            continue
        recommended_titles.append(book_pivot.index[idx])
        # Stop once we have enough recommendations
        if len(recommended_titles) == n_neighbors:
            break

    return recommended_titles

#test
print("Collaborative recommendations for 'A Bend in the Road':")
print(recommend_book('A Bend in the Road'))

Collaborative recommendations for 'A Bend in the Road':
['Exclusive', 'The Cradle Will Fall', 'No Safe Place', 'Family Album', 'Lake Wobegon days', 'Last Man Standing']


#Content Based Recommendation

In [ ]:
#features for recommendation
books['content'] = books['title'] + " " + books['publisher']

#initialize vectorizer
tfidf = TfidfVectorizer(stop_words='english')

#create matrix for content 
tfidf_matrix = tfidf.fit_transform(books['content'])

#check shape
print("TF-IDF Matrix shape:", tfidf_matrix.shape)

TF-IDF Matrix shape: (271360, 82384)


In [ ]:
#now compute cosine similarity-this gives a measure of similarity between two books
#using nearest neighbors

nn = NearestNeighbors(metric='cosine')
nn.fit(tfidf_matrix)

[[     0 205930  54275 189495 193923 111977]]


In [265]:
def get_content_recommendations(book_index, n_neighbors=6):
    distances, indices = nn.kneighbors(tfidf_matrix[book_index], n_neighbors=n_neighbors)
    return distances, indices

#test for a particular book (e.g.index 0)
distances, indices = get_content_recommendations(0, n_neighbors=6)
print("Similar book indices for book index 0:", indices)

Similar book indices for book index 0: [[     0 205930  54275 189495 193923 111977]]


In [268]:
def recommend_books_by_title(book_title, n_neighbors=6):
    #find the index of the book
    if book_title not in books['title'].values:
        return "Book not found in the dataset."
    
    book_index = books.index[books['title'] == book_title][0]
    distances, indices = get_content_recommendations(book_index, n_neighbors=n_neighbors + 1)
    rec_indices = indices[0]

    #map indices to titles and remove the book itself
    recommended_titles = []
    for idx in rec_indices:
        rec_title = books.iloc[idx]['title']
        #skip if the recommended title is the same as the query title
        if rec_title.strip().lower() == book_title.strip().lower():
            continue
        recommended_titles.append(rec_title)
        if len(recommended_titles) == n_neighbors:
            break

    return recommended_titles

#test
print("Content-based recommendations for 'A Bend in the Road':")
print(recommend_books_by_title('A Bend in the Road'))

Content-based recommendations for 'A Bend in the Road':
['Road Show']


In [ ]:
def hybrid_recommendations(book_title, n_neighbors=6, alpha=0.7, beta=0.3):